In [ ]:
run = 504419 # reasonable, low-rate run
# run = 504428 # high-rate run with problems
datadir = f"/Users/tom/cernbox/data/noise/{run}/"
# datadir = f"/eos/user/t/tdietel/data/noise/{run}/"

In [ ]:
import ROOT
%jsroot on

In [ ]:
ROOT.gROOT.LoadMacro("../macros/DataManager.C+")

In [ ]:
# Pythonize iterators
def iter_helper(x):
    x._current = x.begin()
    return x

def next_helper(x):
    if x._current == x.end():
        raise StopIteration
    else: 
        itr = x._current
        x._current += 1
        return itr.__deref__()

ROOT.myrange("o2::trd::Digit, TTreeReaderArray").__iter__ = iter_helper
ROOT.myrange("o2::trd::Digit, TTreeReaderArray").__next__ = next_helper
# ROOT.RawDataSpan.digits.__iter__ = iter_helper
# ROOT.DataManager.digits.__next__ = next_helper

In [ ]:
## This will be nice when pythonization is available (ROOT 6.26 or so...)
# from ROOT import pythonization

# @pythonization('o2::trd::Digit')
# def pythonizor_for_digits(klass):
#     klass.__str__ = lambda d : f'{d.getDetector()}:{d.getROB()}:{d.getMCM():02d}'


# In the meantime, this version is actually just as good
def fmt_digit(d):
    det = d.getDetector()
    r = f'{det:03d}->{det//30:02d}_{(det%30)//5}_{det%6}'
    r += f' {d.getROB()}:{d.getMCM():02d} ch{d.getChannel()}'
    return r

ROOT.o2.trd.Digit.__str__ = fmt_digit
            

In [ ]:
dman = ROOT.DataManager(datadir)
dman.NextTimeFrame()
dman.NextEvent()
dman.NextEvent()

In [ ]:
cnv = ROOT.TCanvas()
adchist = ROOT.TH1F("adchist", "ADC Histogram;ADC;number of entries", 21,-0.5,20.5);
# for i,dig in enumerate(PyDigitRange(dman.Digits())):

event = dman.GetEvent()
for i,dig in enumerate(event.digits):
    if i>2000: break
        
#     if dig.getDetector() != 0:
#         break

    for tb in range(30):
        adchist.Fill(dig.getADC()[tb])
#     print(dig)


In [ ]:
cnv = ROOT.TCanvas()
cnv.SetLogy()
adchist.Draw()
cnv.Draw()